# Computer Vision - Project
imports

In [276]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import h5py
import requests
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf

import h5py, requests, os
import matplotlib.patches as patches
from keras.layers import Conv2D, MaxPooling2D , UpSampling2D ,Conv2DTranspose

In [277]:
# IMG_SIZE = (64, 64)
# IMG_SHAPE = IMG_SIZE + (3,)
# base_model = tf.keras.applications.resnet(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')
# base_model.trainable = False

Constants

In [278]:
#ZIP_PATH = "https://drive.google.com/drive/folders/1jzHYpTwywUYA53nMGHVROSuVO14hEueq?usp=sharing/"
FILE_NAME ="SynthText_train.h5"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
tf.keras.backend.clear_session()


In [279]:
def sort_points(points, center):
    # calculate the angle of each point from the center point
    angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
    # sort the points by angle
    sorted_points = points[np.argsort(angles)]
    return sorted_points

In [280]:
def draw_points(image, points, color=(255, 0, 0), radius=3):
    # create a copy of the image
    img = image.copy()
    # iterate over the points and draw them on the image
    for point in points:
        cv2.circle(img, tuple(map(int, point)), radius, color, -1)
    return img


In [281]:
# def get_bb(img, bbs, indx):
#     x1 = int(bbs[0,0,indx])
#     y1 = int(bbs[1,0,indx])
#     x2 = int(bbs[0,1,indx])
#     y2 = int(bbs[1,1,indx])
#     x3 = int(bbs[0,2,indx])
#     y3 = int(bbs[1,2,indx])
#     x4 = int(bbs[0,3,indx])
#     y4 = int(bbs[1,3,indx])
#     # calculate bounding rectangle
#     top_left_x = max(0, min([x1,x2,x3,x4]))
#     top_left_y = max(0, min([y1,y2,y3,y4]))
#     bot_right_x = max(0, max([x1,x2,x3,x4]))
#     bot_right_y = max(0, max([y1,y2,y3,y4]))
#     points = np.float32([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
#     # create an empty image with the same shape as the input image
#     mask = np.zeros_like((img))
#     # create a list of the bounding box points in the correct format
#     bounding_box = np.array([points], dtype=np.int32)
#     # fill the area inside the bounding box with white
#     cv2.fillPoly(mask, bounding_box, (255, 255, 255))
#     # apply the mask to the image
#     res = cv2.bitwise_and(img, mask)[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
#     #res = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
#     #flipping
#     """"if(x2 < x1):
#         res = cv2.flip(res, 1)
#     if(y2 < y1):
#         res = cv2.flip(res, 0)
#     plt.imshow(res)
#     plt.show()"""
#     return res

In [282]:
def get_bb(img, bbs, indx):
    x1 = int(bbs[0,0,indx])
    y1 = int(bbs[1,0,indx])
    x2 = int(bbs[0,1,indx])
    y2 = int(bbs[1,1,indx])
    x3 = int(bbs[0,2,indx])
    y3 = int(bbs[1,2,indx])
    x4 = int(bbs[0,3,indx])
    y4 = int(bbs[1,3,indx])
    # calculate bounding rectangle
    top_left_x = max(0, min([x1,x2,x3,x4]))
    top_left_y = max(0, min([y1,y2,y3,y4]))
    bot_right_x = max(0, max([x1,x2,x3,x4]))
    bot_right_y = max(0, max([y1,y2,y3,y4]))
    points = np.float32([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    # create an empty image with the same shape as the input image
    mask = np.zeros_like((img))
    # create a list of the bounding box points in the correct format
    bounding_box = np.array([points], dtype=np.int32)
    # fill the area inside the bounding box with white
    cv2.fillPoly(mask, bounding_box, (255, 255, 255))
    inv_mask = cv2.bitwise_not(mask)
    gray_mask = cv2.cvtColor(inv_mask, cv2.COLOR_RGB2GRAY)

    # mask2 = np.zeros_like(img)
    # cv2.fillPoly(mask2, [top_left_y:bot_right_y+1, top_left_x:bot_right_x+1], (255, 255, 255))
    

    res1 = cv2.inpaint(img, gray_mask, 1, cv2.INPAINT_NS)
    # plt.imshow(res1)
    # plt.show()
    # apply the mask to the image
    res = cv2.bitwise_and(img, mask)
    res = cv2.bitwise_and(img, res1)[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    #res = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    #flipping
    """"if(x2 < x1):
        res = cv2.flip(res, 1)
    if(y2 < y1):
        res = cv2.flip(res, 0)
    plt.imshow(res)"""
    plt.show()
    return res

In [283]:
def font_to_num(font):
    if font == b'Alex Brush':
        return 0
    elif font == b'Titillium Web':
        return 1
    elif font == b'Sansation':
        return 2
    elif font == b'Open Sans':
        return 3
    else:
        return 4

In [284]:
# def scale_resize_image(image, size):
#     # image = tf.image.convert_image_dtype(image, tf.float32) # equivalent to dividing image pixels by 255
#     #image = image / 255.0
#     image = tf.image.resize(image, (size, size)) # Resizing the image to 224x224 dimention
#     return image


In [285]:
def prepare_img(img, bbs, index, size):
    cropped = get_bb(img, bbs, index)
    cropped = tf.image.convert_image_dtype(cropped, tf.float32)
    cropped = tf.image.resize(cropped, (size, size))
    cropped = tf.image.rgb_to_grayscale(cropped)
    # cropped = tf.cast(cropped, tf.uint8)
    # cropped = cv2.GaussianBlur(cropped.numpy(), (3,3), 0)
    # print(cropped)
    # cropped = tf.image.convert_image_dtype(cropped, tf.uint8)
    # cropped = cv2.Canny(cropped.numpy(), 0, 3000)

    # cropped = cv2.Canny(cropped, 200, 202)
    # cropped = tf.convert_to_tensor(cropped, dtype=tf.float32)

    #cropped -= cropped.mean()
    # img = tf.ones([size, size, 3], dtype=tf.float32) #(100, 100, 3)
    # mean = tf.constant([1, 2, 3], dtype=tf.float32) # (3)
    # mean = tf.reshape(mean, [1, 1, 3])
    # img_m = img - mean
    #cropped = cv2.cvtColor(np.asarray(cropped, cv2.COLOR_BGR2GRAY))
    # print(cropped)
   
    # 
    # 
    # res = np.squeeze(cropped)
    # res = (cropped * 255).astype(np.uint8)
    
    # res = cv2.GaussianBlur(res, (3,3), 0)
    # # print(res.dtype)
    # # res = np.uint8(res)
    # res = cv2.Canny(res, 0, 3000)
    # # print(res)
    # plt.imshow(cropped, cmap='gray' )
    # plt.show()
    return cropped

In [286]:
def get_data_set(size: int):
    db = h5py.File(FILE_NAME, 'r')
    im_names = list(db['data'].keys())
    train_x = []
    train_y = []
    for i in range(0, len(im_names)-1):
        im = im_names[i]
        img  = db['data'][im][:]
        fonts = db['data'][im].attrs['font']
        txts = db['data'][im].attrs['txt']
        charBBs = db['data'][im].attrs['charBB']
        wordBBs = db['data'][im].attrs['wordBB']
        font_indx = 0 
        char_indx = 0
        for j in range(0, len(txts)):
            cropped = prepare_img(img, wordBBs, j, size)
            train_x.append(cropped)
            train_y.append(font_to_num(fonts[font_indx]))
            font_indx += len(txts[j])
            #plt.imshow(cropped)
            #plt.show()
            for k in range(0, len(txts[j])):
                cropped = prepare_img(img, charBBs, char_indx, size)
                train_x.append(cropped)
                train_y.append(font_to_num(fonts[char_indx]))
                char_indx+=1
                #plt.imshow(cropped)
                #plt.show()
    return train_x, train_y
    
    

In [287]:
train_x, train_y = get_data_set(64)

KeyboardInterrupt: 

In [ ]:
#train_x = np.asarray(train_x, dtype="float") / 255.0
#print(train_x)
#print(train_x)

In [ ]:
#cut the set for train and validation
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
Y_train = np_utils.to_categorical(y_train, 5)
Y_test = np_utils.to_categorical(y_test, 5)

In [ ]:
# model = Sequential([
#   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#   tf.keras.layers.RandomRotation(0.2),
# ])
model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', padding='same', input_shape=(64,64,1)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
# from keras.layers import BatchNormalization
# #  model = Sequential([
# #   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
# #   tf.keras.layers.RandomRotation(0.2),
# # ])
# model = Sequential()
# # Cu Layers 
# model.add(Conv2D(64, kernel_size=(48, 48), activation='relu', input_shape=(64,64,1)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(128, kernel_size=(24, 24), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2DTranspose(128, (24,24), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
# model.add(UpSampling2D(size=(2, 2)))
# model.add(Conv2DTranspose(64, (12,12), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
# model.add(UpSampling2D(size=(2, 2)))
# #Cs Layers
# model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))
# model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))
# model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))
# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(2383,activation='relu'))
# model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [ ]:
X_train = np.array(x_train)
Y_train = np.array(Y_train)
# print(X_train.shape)
# print(Y_train.shape)
# plt.imshow(X_train[2])


In [ ]:
model.fit(X_train, Y_train, batch_size=16, epochs=10, verbose=1)

Epoch 1/10
1896/1896 [==============================] - 11s 5ms/step - loss: 1.6055 - accuracy: 0.2480
Epoch 2/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.5602 - accuracy: 0.3040
Epoch 3/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.5011 - accuracy: 0.3421
Epoch 4/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.4543 - accuracy: 0.3662
Epoch 5/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.4159 - accuracy: 0.3834
Epoch 6/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.3874 - accuracy: 0.3962
Epoch 7/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.3674 - accuracy: 0.4077
Epoch 8/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.3572 - accuracy: 0.4123
Epoch 9/10
1896/1896 [==============================] - 10s 5ms/step - loss: 1.3331 - accuracy: 0.4239
Epoch 10/10
1896/1896 [==============================] - 10s 5ms/step - l

In [ ]:
X_test = np.array(x_test)
#np.random.shuffle(Y_test)
score = model.evaluate(X_test[:1000], Y_test[:1000], verbose=0)

In [ ]:
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 1.2389733791351318 / Test accuracy: 0.4880000054836273


In [ ]:
# plt.imshow(X_train[0])

In [ ]:
# res = model.predict(X_train[:7])
# print(res)

In [ ]:
# print(Y_train[:7])

In [ ]:
#del model 
#tf.keras.backend.clear_session()